In [ ]:
#@title
#changing tensorflow version for compatibility
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

#device configuration
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

# use this command to use gpu
# with tf.device('/device:GPU:0'):

TensorFlow 1.x selected.
1.15.2
Found GPU at: /device:GPU:0


In [ ]:
#draw boxes
import numpy as np
from PIL import Image, ImageDraw
from google.colab import drive


def draw_bbox(img, bbox, text):
    h, w = img.shape[0], img.shape[1]
    x0, y0, x1, y1 = max(bbox[0], 0), max(bbox[1], 0), min(bbox[2], w-1), min(bbox[3], h-1)
    drawed_img = img * 1
    color = np.zeros([x1 - x0 + 1, 3])
    color[:, 1] = np.ones([x1 - x0 + 1]) * 255#Green rectangle
    drawed_img[y0, x0:x1 + 1, :] = color
    drawed_img[y1, x0:x1 + 1, :] = color
    color = np.zeros([y1 - y0 + 1, 3])
    color[:, 1] = np.ones([y1 - y0 + 1]) * 255  # Green rectangle
    drawed_img[y0:y1 + 1, x0, :] = color
    drawed_img[y0:y1 + 1, x1, :] = color
    #type text
    drawed_img = Image.fromarray(np.uint8(drawed_img))
    draw = ImageDraw.Draw(drawed_img)
    x = int(bbox[0])
    y = int(bbox[1])
    draw.text((x, y), text)
    drawed_img = np.array(drawed_img)
    return drawed_img

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar

--2020-11-03 17:48:51--  http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar
Resolving host.robots.ox.ac.uk (host.robots.ox.ac.uk)... 129.67.94.152
Connecting to host.robots.ox.ac.uk (host.robots.ox.ac.uk)|129.67.94.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 460032000 (439M) [application/x-tar]
Saving to: ‘VOCtrainval_06-Nov-2007.tar’

VOCtrainval_06-Nov- 100%[===================>] 438.72M  9.07MB/s    in 53s     

2020-11-03 17:49:44 (8.31 MB/s) - ‘VOCtrainval_06-Nov-2007.tar’ saved [460032000/460032000]



In [ ]:
!mkdir /content/VOC
!tar -xvf  '/content/VOCtrainval_06-Nov-2007.tar' -C '/content/VOC'

In [ ]:
#networks VGG-16

import tensorflow as tf

import numpy as np

#convolutional layer
def conv(inputs, w, b, trainable=False):
    w = tf.Variable(initial_value=w, trainable=trainable)
    b = tf.Variable(initial_value=b, trainable=trainable)
    inputs = tf.nn.conv2d(inputs, w, [1, 1, 1, 1], "SAME")
    return tf.nn.bias_add(inputs, b)
#max pooling layer
def max_pooling(inputs):
    return tf.nn.max_pool(inputs, [1, 2, 2, 1], [1, 2, 2, 1], "SAME")
#relu
def relu(inputs):
    return tf.nn.relu(inputs)
#fully connected layer
def vgg_fully_connected(name, inputs, init_W, init_b):
    with tf.variable_scope(name):
        inputs = tf.layers.flatten(inputs)
        W = tf.Variable(initial_value=init_W)
        b = tf.Variable(initial_value=init_b)
        inputs = tf.matmul(inputs, W)
        inputs = tf.nn.bias_add(inputs, b)
    return inputs

def conv_(name, inputs, nums_out, k_size, strides, padding="SAME"):
    nums_in = inputs.shape[-1]
    with tf.variable_scope(name):
        W = tf.get_variable("W", [k_size, k_size, nums_in, nums_out], initializer=tf.truncated_normal_initializer(stddev=0.02))
        b = tf.get_variable("b", [nums_out], initializer=tf.constant_initializer([0.]))
        inputs = tf.nn.conv2d(inputs, W, [1, strides, strides, 1], padding=padding)
        inputs = tf.nn.bias_add(inputs, b)
    return inputs

def fc(name, inputs, nums_out):
    nums_in = inputs.shape[-1]
    with tf.variable_scope(name):
        W = tf.get_variable("W", [nums_in, nums_out], initializer=tf.truncated_normal_initializer(stddev=0.02))
        b = tf.get_variable("b", [nums_out], initializer=tf.constant_initializer([0.]))
        inputs = tf.matmul(inputs, W)
    return tf.nn.bias_add(inputs, b)


def vgg16(inputs):
    inputs = tf.reverse(inputs, [-1]) - tf.constant([103.939, 116.779, 123.68])
    inputs /= 255.0
    para = np.load("/content/drive/My Drive/vgg16.npy.npy（副本）", encoding="latin1",allow_pickle=True).item()
    inputs = relu(conv(inputs, para["conv1_1"][0], para["conv1_1"][1]))
    inputs = relu(conv(inputs, para["conv1_2"][0], para["conv1_2"][1]))
    inputs = max_pooling(inputs)
    inputs = relu(conv(inputs, para["conv2_1"][0], para["conv2_1"][1]))
    inputs = relu(conv(inputs, para["conv2_2"][0], para["conv2_2"][1]))
    inputs = max_pooling(inputs)
    inputs = relu(conv(inputs, para["conv3_1"][0], para["conv3_1"][1], True))
    inputs = relu(conv(inputs, para["conv3_2"][0], para["conv3_2"][1], True))
    inputs = relu(conv(inputs, para["conv3_3"][0], para["conv3_3"][1], True))
    inputs = max_pooling(inputs)
    inputs = relu(conv(inputs, para["conv4_1"][0], para["conv4_1"][1], True))
    inputs = relu(conv(inputs, para["conv4_2"][0], para["conv4_2"][1], True))
    inputs = relu(conv(inputs, para["conv4_3"][0], para["conv4_3"][1], True))
    inputs = max_pooling(inputs)
    inputs = relu(conv(inputs, para["conv5_1"][0], para["conv5_1"][1], True))
    inputs = relu(conv(inputs, para["conv5_2"][0], para["conv5_2"][1], True))
    inputs = relu(conv(inputs, para["conv5_3"][0], para["conv5_3"][1], True))
    inputs = max_pooling(inputs)
    inputs = tf.layers.flatten(inputs)
    inputs = fc("full", inputs, 512)
    inputs = fc("logits", inputs, 7*7*30)
    inputs = tf.reshape(inputs, [-1, 7, 7, 30])
    return tf.sigmoid(inputs)

In [ ]:
#loss_YOLO




def yolo_loss(prediction, labels, H=448, W=448, S=7):
    #prediction: batchsize x 7 x 7 x 30, bboxes: batchsize x 7 x 7 x (0:8), confidences: batchsize x 7 x 7 x (8:10), class: batchsize x 7 x 7 x (10:30)
    #labels: batchsize x 7 x 7 x 25, response_mask: batchsize x 7 x 7 x (0:1), bbox: batchsize x 7 x 7 x (1:5), class: batchsize: batchsize x 7 x 7 x (5:25)
    pred_bboxes = prediction[:, :, :, 0:8]
    pred_bboxes = tf.reshape(pred_bboxes, [-1, 7, 7, 2, 4])
    pred_confidence = prediction[:, :, :, 8:10]
    pred_class = prediction[:, :, :, 10:]

    #groundtruth
    label_mask = labels[:, :, :, 0:1]
    label_bboxes = labels[:, :, :, 1:5]
    label_bboxes = tf.tile(label_bboxes, multiples=[1, 1, 1, 2])
    label_bboxes = tf.reshape(label_bboxes, [-1, 7, 7, 2, 4])
    label_class = labels[:, :, :, 5:]

    #prediction normalized offset -> [c_x, c_y, h, w]
    cell_h = H / S #normalising the height
    cell_w = W / S #normalising the width
    temp = tf.constant([[0., 1., 2., 3., 4., 5., 6.]])
    temp = tf.tile(temp, multiples=[7, 1])
    col = temp[tf.newaxis, :, :, tf.newaxis, tf.newaxis]#dimension 7 x 7 -> 1 x 7 x 7 x 1 x 1
    row = tf.transpose(col, perm=[0, 2, 1, 3, 4])
    pred_bboxes_original = tf.concat([(pred_bboxes[:, :, :, :, 0:1] + col) * cell_w,
                                      (pred_bboxes[:, :, :, :, 1:2] + row) * cell_h,
                                       tf.square(pred_bboxes[:, :, :, :, 2:3]) * H,
                                       tf.square(pred_bboxes[:, :, :, :, 3:4]) * W], axis=-1)
    label_bboxes_original = tf.concat([(label_bboxes[:, :, :, :, 0:1] + col) * cell_w,
                                      (label_bboxes[:, :, :, :, 1:2] + row) * cell_h,
                                      tf.square(label_bboxes[:, :, :, :, 2:3]) * H,
                                      tf.square(label_bboxes[:, :, :, :, 3:4]) * W], axis=-1)
    iou = cal_iou(pred_bboxes_original, label_bboxes_original)#input: batchsize x 7 x 7 x 2 x 4, output: batchsize x 7 x 7 x 2
    max_iou = tf.reduce_max(iou, axis=-1, keep_dims=True)#output: batchsize x 7 x 7 x 1
    mask_obj = tf.cast(tf.greater_equal(iou, max_iou), dtype=tf.float32) * label_mask
    loss_bboxes = tf.reduce_mean(tf.reduce_sum(tf.square(pred_bboxes - label_bboxes) * mask_obj[:, :, :, :, tf.newaxis], axis=[1, 2, 3, 4]))
    loss_confidence_obj = tf.reduce_mean(tf.reduce_sum(tf.square(pred_confidence - 1) * mask_obj, axis=[1, 2, 3]))
    loss_confidence_noobj = tf.reduce_mean(tf.reduce_sum(tf.square(pred_confidence) * (1 - mask_obj), axis=[1, 2, 3]))
    loss_class = tf.reduce_mean(tf.reduce_sum(tf.square(pred_class - label_class) * label_mask, axis=[1, 2, 3]))
    loss = 5 * loss_bboxes + loss_confidence_obj + 0.5 * loss_confidence_noobj + loss_class #lamda values for coord and noobj respectively are 5 and 0.5
    return loss, loss_bboxes, loss_confidence_obj, loss_confidence_noobj, loss_class

def cal_iou(bboxes1, bboxes2):
    #bboxes: [batchsize, 7, 7, 2, 4] with [center_x, center_y, h, w]
    cx, cx_ = bboxes1[:, :, :, :, 0], bboxes2[:, :, :, :, 0]
    cy, cy_ = bboxes1[:, :, :, :, 1], bboxes2[:, :, :, :, 1]
    h, h_ = bboxes1[:, :, :, :, 2], bboxes2[:, :, :, :, 2]
    w, w_ = bboxes1[:, :, :, :, 3], bboxes2[:, :, :, :, 3]
    x1, x1_ = cx - w / 2, cx_ - w_ / 2
    x2, x2_ = cx + w / 2, cx_ + w_ / 2
    y1, y1_ = cy - h / 2, cy_ - h_ / 2
    y2, y2_ = cy + h / 2, cy_ + h_ / 2
    x_inter1 = tf.maximum(x1, x1_)
    x_inter2 = tf.minimum(x2, x2_)
    y_inter1 = tf.maximum(y1, y1_)
    y_inter2 = tf.minimum(y2, y2_)
    h_inter = tf.maximum(0., y_inter2 - y_inter1)
    w_inter = tf.maximum(0., x_inter2 - x_inter1)
    area_inter = h_inter * w_inter
    area_union = h * w + h_ * w_ - area_inter
    iou = area_inter / area_union
    return iou

if __name__ == "__main__":
    prediction = tf.placeholder(tf.float32, [None, 7, 7, 30])
    labels = tf.placeholder(tf.float32, [None, 7, 7, 25])
    yolo_loss(prediction, labels)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [ ]:
import xml.etree.cElementTree as ET
import numpy as np
import os
from PIL import Image
import scipy.misc as misc
import scipy.io as sio


OBJECT_NAMES = ["tvmonitor", "train", "sofa", "sheep", "cat", "chair", "bottle", "motorbike", "boat", "bird",
                   "person", "aeroplane", "dog", "pottedplant", "cow", "bus", "diningtable", "horse", "bicycle", "car"]
EPSILON = 1e-8

def read_xml(path):
    tree = ET.parse(path)
    root = tree.getroot()
    objects = root.findall("object")
    imgname = root.find("filename").text
    gt_bbox = np.zeros([objects.__len__(), 4], dtype=np.int32)
    name_bbox = []
    for i, obj in enumerate(objects):
        objectname = obj.find("name").text
        bbox = np.zeros([4], dtype=np.int32)
        xmin = int(obj.find("bndbox").find("xmin").text)
        ymin = int(obj.find("bndbox").find("ymin").text)
        xmax = int(obj.find("bndbox").find("xmax").text)
        ymax = int(obj.find("bndbox").find("ymax").text)
        bbox[0], bbox[1], bbox[2], bbox[3] = xmin, ymin, xmax, ymax
        name_bbox.append(objectname)
        gt_bbox[i, :] = bbox

    return imgname, gt_bbox, name_bbox

def cal_iou_1(bbox1, bbox2):
    #bbox = [x1, y1, x2, y2]
    x1, y1, x1_, y1_ = bbox1[0], bbox1[1], bbox1[2], bbox1[3]
    x2, y2, x2_, y2_ = bbox2[0], bbox2[1], bbox2[2], bbox2[3]
    x0 = max(x1, x2)
    y0 = max(y1, y2)
    x0_ = min(x1_, x2_)
    y0_ = min(y1_, y2_)

    if x0 >= x0_ or y0 >= y0_:
        iou = 0
    else:
        inter_area = (x0_ - x0) * (y0_ - y0)
        bbox1_area = (x1_ - x1) * (y1_ - y1)
        bbox2_area = (x2_ - x2) * (y2_ - y2)
        union_area = bbox1_area + bbox2_area - inter_area

        iou = inter_area / union_area
    return iou

def ToScaleImg(img, tar_h, tar_w, raw_bboxes):
    h, w = img.shape[0], img.shape[1]
    nums_bbox = raw_bboxes.shape[0]
    tar_bboxes = np.zeros_like(raw_bboxes)
    for i in range(nums_bbox):
        bbox = raw_bboxes[i]
        x0, y0, x1, y1 = bbox[0], bbox[1], bbox[2], bbox[3]
        x0 = tar_w / w * x0
        x1 = tar_w / w * x1
        y0 = tar_h / h * y0
        y1 = tar_h / h * y1
        tar_bboxes[i, 0], tar_bboxes[i, 1] = x0, y0
        tar_bboxes[i, 2], tar_bboxes[i, 3] = x1, y1
    scaled_img = misc.imresize(img, [tar_h, tar_w])
    return scaled_img, tar_bboxes


def read_batch(img_path, xml_path, batch_size, img_h=448, img_w=448):
    xml_lists = os.listdir(xml_path)
    nums = xml_lists.__len__()
    rand_idx = np.random.randint(0, nums, [batch_size])
    batch_bboxes = np.zeros([batch_size, 7, 7, 4])
    batch_classes = np.zeros([batch_size, 7, 7, 20])
    batch_img = np.zeros([batch_size, img_h, img_w, 3])
    cell_h = img_h / 7
    cell_w = img_w / 7
    for j in range(batch_size):
        imgname, gt_bbox, name_bbox = read_xml(xml_path + xml_lists[rand_idx[j]])
        img = np.array(Image.open(img_path + imgname))
        scaled_img, scaled_bbox = ToScaleImg(img, img_h, img_w, gt_bbox)
        batch_img[j, :, :, :] = scaled_img
        for i in range(scaled_bbox.shape[0]):
            c_x = (scaled_bbox[i, 0] + scaled_bbox[i, 2]) / 2
            c_y = (scaled_bbox[i, 1] + scaled_bbox[i, 3]) / 2
            h = scaled_bbox[i, 3] - scaled_bbox[i, 1]
            w = scaled_bbox[i, 2] - scaled_bbox[i, 0]
            col = int(c_x // cell_w)
            row = int(c_y // cell_h)
            offset_x = c_x / cell_w - col
            offset_y = c_y / cell_h - row
            offset_h = np.sqrt(h / img_h)
            offset_w = np.sqrt(w / img_w)
            batch_bboxes[j, row, col, 0], batch_bboxes[j, row, col, 1] = offset_x, offset_y
            batch_bboxes[j, row, col, 2], batch_bboxes[j, row, col, 3] = offset_h, offset_w
            index = OBJECT_NAMES.index(name_bbox[i])
            batch_classes[j, row, col, index] = 1
    batch_labels = np.zeros([batch_size, 7, 7, 25])
    batch_response = np.sum(batch_classes, axis=-1, keepdims=True)
    batch_labels[:, :, :, 0:1] = batch_response
    batch_labels[:, :, :, 1:5] = batch_bboxes
    batch_labels[:, :, :, 5:] = batch_classes
    return batch_img, batch_labels

def img2mat(imgpath, xmlpath):
    filenames = os.listdir(xmlpath)
    nums = filenames.__len__()
    imgs = np.zeros([nums, 448, 448, 3], dtype=np.uint8)
    xml = []
    class_name = []
    for idx, filename in enumerate(filenames):
        imgname, gt_bbox, name_bbox = read_xml(xmlpath + filename)
        img = np.array(Image.open(imgpath + imgname))
        scaled_img, scaled_bbox = ToScaleImg(img, 448, 448, gt_bbox)
        imgs[idx, :, :, :] = scaled_img
        xml.append(scaled_bbox)
        class_name.append(name_bbox)
        print(idx)
    sio.savemat("pascal.mat", {"imgs": imgs, "bboxes": xml, "class": class_name})


In [ ]:
#training model




BATCH_SIZE = 64
img_path = "/content/VOC/VOCdevkit/VOC2007/JPEGImages"
xml_path = "/content/VOC/VOCdevkit/VOC2007/Annotations"


def train():
    inputs = tf.placeholder(tf.float32, [None, 448, 448, 3])
    labels = tf.placeholder(tf.float32, [None, 7, 7, 25])
    prediction = vgg16(inputs)
    loss, loss_bboxes, loss_confidence_obj, loss_confidence_noobj, loss_class = yolo_loss(prediction, labels)
    Opt = tf.train.MomentumOptimizer(1e-3, momentum=0.9).minimize(loss)
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    # saver.restore(sess, "./save_para/.\\model.ckpt")
    for i in range(10000):
        batch_img, batch_labels = read_batch(img_path, xml_path, BATCH_SIZE)
        sess.run(Opt, feed_dict={inputs: batch_img, labels: batch_labels})
        [LOSS, b, o, n, c] = sess.run([loss, loss_bboxes, loss_confidence_obj, loss_confidence_noobj, loss_class], feed_dict={inputs: batch_img, labels: batch_labels})
        print("Iteration: %d, Loss: %f, loss_bbox: %f, loss_confi_obj: %f, loss_confi_noobj: %f, loss_class: %f"%(i, LOSS, b, o, n, c))
        if i % 500 == 0:
            saver.save(sess, "./save_para/model.ckpt")


if __name__ == "__main__":
    train()

In [ ]:
#testing YOLO


#
# import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

BATCH_SIZE = 1
img_path = "/content/VOC/VOCdevkit/VOC2007/JPEGImages"
xml_path = "/content/VOC/VOCdevkit/VOC2007/Annotations"

OBJECT_NAMES = ["tvmonitor", "train", "sofa", "sheep", "cat", "chair", "bottle", "motorbike", "boat", "bird",
                   "person", "aeroplane", "dog", "pottedplant", "cow", "bus", "diningtable", "horse", "bicycle", "car"]

def norm2bbox(norm_bbox, H=448, W=448, S=7):
    temp = tf.constant([[0., 1., 2., 3., 4., 5., 6.]])
    col = tf.tile(temp, multiples=[7, 1])[tf.newaxis, :, :, tf.newaxis, tf.newaxis]
    row = tf.transpose(col, perm=[0, 2, 1, 3, 4])
    x, y = norm_bbox[:, :, :, :, 0:1], norm_bbox[:, :, :, :, 1:2]
    h, w = norm_bbox[:, :, :, :, 2:3], norm_bbox[:, :, :, :, 3:4]
    x = (x + col) * (W / S)
    y = (y + row) * (H / S)
    h = tf.square(h) * H
    w = tf.square(w) * W
    x1 = x - w/2
    y1 = y - h/2
    x2 = x + w/2
    y2 = y + h/2
    coord = tf.concat([x1, y1, x2, y2], axis=-1)
    return coord

def select_bbox_again(indx_class, scores_class, pred_bboxes):
    #indx_class: the index after non-maximun suppression, [20, 5]
    #scores_class: confidence * class_confidence, [20, 98]
    #pred_bboxes: [98, 4]
    mask = np.zeros_like(scores_class)
    for i in range(20):
        for j in range(5):
            mask[i, indx_class[i, j]] = 1
    scores_class[scores_class < 0.2] = 0
    scores_class *= mask
    max_score = np.max(scores_class, axis=0)
    indx_bboxes = np.arange(0, 98)
    indx_bboxes = indx_bboxes[max_score > 0]
    class_indx = np.argmax(scores_class, axis=0)
    bbox_indx = class_indx[max_score > 0]
    return pred_bboxes[indx_bboxes], bbox_indx

def test(img):
    inputs = tf.placeholder(tf.float32, [1, 448, 448, 3])
    prediction = vgg16(inputs)
    pred_bboxes = prediction[:, :, :, :8]#[1, 7, 7, 8]
    pred_bboxes = tf.reshape(pred_bboxes, [1, 7, 7, 2, 4])#[1, 7, 7, 2, 4]
    pred_confidence = prediction[:, :, :, 8:10]#[1, 7, 7, 2]
    pred_class = prediction[:, :, :, 10:]#[1, 7, 7, 20]
    pred_bboxes = norm2bbox(pred_bboxes)#[1, 7, 7, 2, 4]
    class_confidences = tf.split(pred_class, num_or_size_splits=20, axis=-1)
    pred_bboxes = tf.reshape(pred_bboxes, [-1, 4])#reshaping to feed into non-max suppression unit
    indx_class = []
    scores_class = []
    for class_confidence in class_confidences:
        scores = pred_confidence * class_confidence#[1, 7, 7, 2]
        scores = tf.reshape(scores, [-1])
        indx = tf.image.non_max_suppression(pred_bboxes, scores, 5)
        indx_class.append(indx[tf.newaxis, :])
        scores_class.append(scores[tf.newaxis, :])
    indx_class = tf.concat(indx_class, axis=0)
    scores_class = tf.concat(scores_class, axis=0)
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    #saver = tf.train.Saver()
    #saver.restore(sess, "./save_para/.\\model.ckpt")

    INDX_CLASS, PRED_BBOXES, SCORES_CLASS = sess.run([indx_class, pred_bboxes, scores_class], feed_dict={inputs: img[np.newaxis, :, :, :]})
    bboxes, class_num = select_bbox_again(INDX_CLASS, SCORES_CLASS, PRED_BBOXES)
    for i in range(bboxes.shape[0]):
        try:
            img = draw_bbox(img, np.int32(bboxes[i]), OBJECT_NAMES[class_num[i]])
        except:
            continue
    Image.fromarray(np.uint8(img)).show()



if __name__ == "__main__":
    img = np.array(Image.open("/content/drive/My Drive/keanu.jpg").resize([448, 448]))
    test(img)